# Developing, Training, and Deploying a TensorFlow model on Google Cloud Platform (completely within Jupyter)


In Chapter 9 of [Data Science on the Google Cloud Platform](http://shop.oreilly.com/product/0636920057628.do), I trained a TensorFlow Estimator model to predict flight delays.

In this notebook, we'll modernize the workflow:
* Use eager mode for TensorFlow development
* Use tf.data to write the input pipeline
* Run the notebook as-is on Cloud using Deep Learning VM or Kubeflow pipelines
* Deploy the trained model to AI Platform as a web service

The combination of eager mode, tf.data and DLVM/KFP makes this workflow a lot easier.
We don't need to deal with Python packages or Docker containers.

In [1]:
# change these to try this notebook out
# In "production", these will be replaced by the parameters passed to papermill
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'
DEVELOP_MODE = True
NBUCKETS = 5

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Creating the input data pipeline

In [4]:
DATA_BUCKET = "gs://cloud-training-demos/flights/chapter8/output/"
TRAIN_DATA_PATTERN = DATA_BUCKET + "train*"
VALID_DATA_PATTERN = DATA_BUCKET + "test*"

In [5]:
!gsutil ls $DATA_BUCKET

gs://cloud-training-demos/flights/chapter8/output/delays.csv
gs://cloud-training-demos/flights/chapter8/output/testFlights-00000-of-00007.csv
gs://cloud-training-demos/flights/chapter8/output/testFlights-00001-of-00007.csv
gs://cloud-training-demos/flights/chapter8/output/testFlights-00002-of-00007.csv
gs://cloud-training-demos/flights/chapter8/output/testFlights-00003-of-00007.csv
gs://cloud-training-demos/flights/chapter8/output/testFlights-00004-of-00007.csv
gs://cloud-training-demos/flights/chapter8/output/testFlights-00005-of-00007.csv
gs://cloud-training-demos/flights/chapter8/output/testFlights-00006-of-00007.csv
gs://cloud-training-demos/flights/chapter8/output/trainFlights-00000-of-00007.csv
gs://cloud-training-demos/flights/chapter8/output/trainFlights-00001-of-00007.csv
gs://cloud-training-demos/flights/chapter8/output/trainFlights-00002-of-00007.csv
gs://cloud-training-demos/flights/chapter8/output/trainFlights-00003-of-00007.csv
gs://cloud-training-demos/flights/chapter8/o

### Use tf.data to read the CSV files

In [6]:
import os, json, math
import numpy as np
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.0.0-alpha0


In [7]:
CSV_COLUMNS  = ('ontime,dep_delay,taxiout,distance,avg_dep_delay,avg_arr_delay' + \
                ',carrier,dep_lat,dep_lon,arr_lat,arr_lon,origin,dest').split(',')
LABEL_COLUMN = 'ontime'
DEFAULTS     = [[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],\
                ['na'],[0.0],[0.0],[0.0],[0.0],['na'],['na']]

def load_dataset(pattern):
  return tf.data.experimental.make_csv_dataset(pattern, 1, CSV_COLUMNS, DEFAULTS)

In [8]:
if DEVELOP_MODE:
    dataset = load_dataset(TRAIN_DATA_PATTERN)
    for n, data in enumerate(dataset):
        numpy_data = {k: v.numpy() for k, v in data.items()} # .numpy() works only in eager mode
        print(numpy_data)
        if n>3: break

{'arr_lon': array([-90.075836], dtype=float32), 'dest': array([b'JAN'], dtype=object), 'dep_delay': array([-1.], dtype=float32), 'arr_lat': array([32.31111], dtype=float32), 'taxiout': array([20.], dtype=float32), 'dep_lat': array([33.636665], dtype=float32), 'avg_dep_delay': array([33.790806], dtype=float32), 'carrier': array([b'EV'], dtype=object), 'ontime': array([1.], dtype=float32), 'dep_lon': array([-84.42778], dtype=float32), 'origin': array([b'ATL'], dtype=object), 'avg_arr_delay': array([-4.], dtype=float32), 'distance': array([341.], dtype=float32)}
{'arr_lon': array([-90.075836], dtype=float32), 'dest': array([b'JAN'], dtype=object), 'dep_delay': array([-3.], dtype=float32), 'arr_lat': array([32.31111], dtype=float32), 'taxiout': array([19.], dtype=float32), 'dep_lat': array([32.896946], dtype=float32), 'avg_dep_delay': array([27.40805], dtype=float32), 'carrier': array([b'MQ'], dtype=object), 'ontime': array([1.], dtype=float32), 'dep_lon': array([-97.038055], dtype=float32

In [9]:
%%writefile example_input.json
{"dep_delay": 14.0, "taxiout": 13.0, "distance": 319.0, "avg_dep_delay": 25.863039, "avg_arr_delay": 27.0, "carrier": "WN", "dep_lat": 32.84722, "dep_lon": -96.85167, "arr_lat": 31.9425, "arr_lon": -102.20194, "origin": "DAL", "dest": "MAF"}
{"dep_delay": -9.0, "taxiout": 21.0, "distance": 301.0, "avg_dep_delay": 41.050808, "avg_arr_delay": -7.0, "carrier": "EV", "dep_lat": 29.984444, "dep_lon": -95.34139, "arr_lat": 27.544167, "arr_lon": -99.46167, "origin": "IAH", "dest": "LRD"}

Overwriting example_input.json


In [10]:
def features_and_labels(features):
  label = features.pop('ontime') # this is what we will train for
  return features, label

def prepare_dataset(pattern, batch_size, truncate=None, mode=tf.estimator.ModeKeys.TRAIN):
  dataset = load_dataset(pattern)
  dataset = dataset.map(features_and_labels)
  dataset = dataset.cache()
  if mode == tf.estimator.ModeKeys.TRAIN:
    dataset = dataset.shuffle(1000)
    dataset = dataset.repeat()
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(1)
  if truncate is not None:
    dataset = dataset.take(truncate)
  return dataset

if DEVELOP_MODE:
    print("Calling prepare")
    one_item = prepare_dataset(TRAIN_DATA_PATTERN, batch_size=5, truncate=1)
    print(list(one_item)) # should print one batch of 2 items

Calling prepare
[(OrderedDict([('dep_delay', <tf.Tensor: id=345, shape=(5, 1), dtype=float32, numpy=
array([[ 0.],
       [-1.],
       [-5.],
       [-6.],
       [-3.]], dtype=float32)>), ('taxiout', <tf.Tensor: id=351, shape=(5, 1), dtype=float32, numpy=
array([[20.],
       [50.],
       [20.],
       [17.],
       [14.]], dtype=float32)>), ('distance', <tf.Tensor: id=349, shape=(5, 1), dtype=float32, numpy=
array([[145.],
       [119.],
       [119.],
       [145.],
       [106.]], dtype=float32)>), ('avg_dep_delay', <tf.Tensor: id=343, shape=(5, 1), dtype=float32, numpy=
array([[26.245787],
       [26.036997],
       [24.971495],
       [24.760445],
       [25.268755]], dtype=float32)>), ('avg_arr_delay', <tf.Tensor: id=342, shape=(5, 1), dtype=float32, numpy=
array([[  0. ],
       [ 15. ],
       [  9.5],
       [ -9. ],
       [-16. ]], dtype=float32)>), ('carrier', <tf.Tensor: id=344, shape=(5, 1), dtype=string, numpy=
array([[b'EV'],
       [b'OO'],
       [b'OO'],
       [b

## Create TensorFlow wide-and-deep model

We'll create feature columns, and do some discretization and feature engineering.
See the book for details.

In [17]:
import tensorflow.feature_column as fc

real = {
    colname : fc.numeric_column(colname) \
          for colname in \
            ('dep_delay,taxiout,distance,avg_dep_delay,avg_arr_delay' +
             ',dep_lat,dep_lon,arr_lat,arr_lon').split(',')
}
sparse = {
      'carrier': fc.categorical_column_with_vocabulary_list('carrier',
                  vocabulary_list='AS,VX,F9,UA,US,WN,HA,EV,MQ,DL,OO,B6,NK,AA'.split(',')),
      'origin' : fc.categorical_column_with_hash_bucket('origin', hash_bucket_size=1000),
      'dest'   : fc.categorical_column_with_hash_bucket('dest', hash_bucket_size=1000)
}

inputs = {
    colname : tf.keras.layers.Input(name=colname, shape=(), dtype='float32') \
          for colname in real.keys()
}
inputs.update({
    colname : tf.keras.layers.Input(name=colname, shape=(), dtype='string') \
          for colname in sparse.keys()
})

### Feature engineering

In [18]:
latbuckets = np.linspace(20.0, 50.0, NBUCKETS).tolist()  # USA
lonbuckets = np.linspace(-120.0, -70.0, NBUCKETS).tolist() # USA
disc = {}
disc.update({
       'd_{}'.format(key) : fc.bucketized_column(real[key], latbuckets) \
          for key in ['dep_lat', 'arr_lat']
})
disc.update({
       'd_{}'.format(key) : fc.bucketized_column(real[key], lonbuckets) \
          for key in ['dep_lon', 'arr_lon']
})

# cross columns that make sense in combination
sparse['dep_loc'] = fc.crossed_column([disc['d_dep_lat'], disc['d_dep_lon']], NBUCKETS*NBUCKETS)
sparse['arr_loc'] = fc.crossed_column([disc['d_arr_lat'], disc['d_arr_lon']], NBUCKETS*NBUCKETS)
sparse['dep_arr'] = fc.crossed_column([sparse['dep_loc'], sparse['arr_loc']], NBUCKETS ** 4)
sparse['ori_dest'] = fc.crossed_column(['origin', 'dest'], hash_bucket_size=1000)

# embed all the sparse columns
embed = {
       'embed_{}'.format(colname) : fc.embedding_column(col, 10) \
          for colname, col in sparse.items()
}
real.update(embed)

# one-hot encode the sparse columns
sparse = {
    colname : fc.indicator_column(col) \
          for colname, col in sparse.items()
}

if DEVELOP_MODE:
    print(sparse.keys())
    print(real.keys())

dict_keys(['carrier', 'dest', 'origin', 'dep_loc', 'arr_loc', 'ori_dest', 'dep_arr'])
dict_keys(['arr_lon', 'dep_delay', 'arr_lat', 'taxiout', 'embed_dep_arr', 'dep_lat', 'embed_dep_loc', 'embed_origin', 'embed_ori_dest', 'embed_carrier', 'embed_dest', 'distance', 'avg_dep_delay', 'dep_lon', 'embed_arr_loc', 'avg_arr_delay'])


## Train the model and evaluate once in a while

Also checkpoint

In [19]:
model_dir='gs://{}/flights/trained_model'.format(BUCKET)
os.environ['OUTDIR'] = model_dir  # needed for deployment
print('Writing trained model to {}'.format(model_dir))

Writing trained model to gs://cloud-training-demos-ml/flights/trained_model


In [20]:
!gsutil -m rm -rf $OUTDIR

CommandException: 1 files/objects could not be removed.


In [21]:
# Build a wide-and-deep model.
def wide_and_deep_classifier(inputs, linear_feature_columns, dnn_feature_columns, dnn_hidden_units):
    deep = tf.keras.layers.DenseFeatures(dnn_feature_columns)(inputs)
    for numnodes in dnn_hidden_units:
        deep = tf.keras.layers.Dense(numnodes, activation='relu')(deep)        
    wide = tf.keras.layers.DenseFeatures(linear_feature_columns)(inputs)
    both = tf.keras.layers.concatenate([deep, wide])
    output = tf.keras.layers.Dense(1, activation='sigmoid')(both)
    model = tf.keras.Model(inputs, output)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model
    
model = wide_and_deep_classifier(
    inputs,
    linear_feature_columns = sparse.values(),
    dnn_feature_columns = real.values(),
    dnn_hidden_units = [64, 32])
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
arr_lat (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
arr_lon (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
avg_arr_delay (InputLayer)      [(None,)]            0                                            
__________________________________________________________________________________________________
avg_dep_delay (InputLayer)      [(None,)]            0                                            
____________________________________________________________________________________________

In [22]:
# training and evaluation dataset
train_batch_size = 64
eval_batch_size = 100 if DEVELOP_MODE else 10000
num_steps = 10 if DEVELOP_MODE else (1000000 // train_batch_size)
train_dataset = prepare_dataset(TRAIN_DATA_PATTERN, train_batch_size)
eval_dataset = prepare_dataset(VALID_DATA_PATTERN, eval_batch_size, eval_batch_size*10, tf.estimator.ModeKeys.EVAL)

model.fit(train_dataset, 
          validation_data=eval_dataset,
          epochs=num_steps, steps_per_epoch=1)

Epoch 1/10
1/1 [==============================] - 65s 65s/step - loss: 1.9135 - accuracy: 0.1875 - val_loss: 1.5502 - val_accuracy: 0.1546
Epoch 2/10
1/1 [==============================] - 65s 65s/step - loss: 0.9439 - accuracy: 0.2344 - val_loss: 0.5983 - val_accuracy: 0.7562
Epoch 3/10
1/1 [==============================] - 64s 64s/step - loss: 0.5457 - accuracy: 0.8438 - val_loss: 0.4300 - val_accuracy: 0.8526
Epoch 4/10
1/1 [==============================] - 64s 64s/step - loss: 0.3936 - accuracy: 0.8594 - val_loss: 0.5586 - val_accuracy: 0.8514
Epoch 5/10
1/1 [==============================] - 64s 64s/step - loss: 0.5546 - accuracy: 0.7969 - val_loss: 0.6831 - val_accuracy: 0.8512
Epoch 6/10
1/1 [==============================] - 65s 65s/step - loss: 0.5631 - accuracy: 0.8594 - val_loss: 0.7669 - val_accuracy: 0.8511
Epoch 7/10
1/1 [==============================] - 64s 64s/step - loss: 0.2470 - accuracy: 0.9375 - val_loss: 0.8354 - val_accuracy: 0.8510
Epoch 8/10
1/1 [===========

## Deploy the trained model

In [25]:
%%bash
model_dir=$(gsutil ls ${OUTDIR}/export/exporter | tail -1)
echo $model_dir
saved_model_cli show --dir ${model_dir} --all

gs://cloud-training-demos-ml/flights/trained_model/export/exporter/1553294283/

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['arr_lat'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_7:0
    inputs['arr_lon'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_8:0
    inputs['avg_arr_delay'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_4:0
    inputs['avg_dep_delay'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_3:0
    inputs['carrier'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: Placeholder_9:0
    inputs['dep_delay'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder:0
    inputs['dep_lat'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
     

In [ ]:
%%bash
MODEL_NAME="flights"
MODEL_VERSION="kfp"
TFVERSION="2.0"
MODEL_LOCATION=$(gsutil ls ${OUTDIR}/export/exporter | tail -1)
echo "Run these commands one-by-one (the very first time, you'll create a model and then create a version)"
#yes | gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

In [ ]:
!gcloud ml-engine predict --model=flights --version=kfp --json-instances=example_input.json

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License